Sentiment Analysis

In [4]:
#ALL HEADER FILES

import nltk
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.models import Model
from keras.layers import Input,Flatten, Dense, Embedding, RNN, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, concatenate, Lambda
from keras import optimizers
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from keras import optimizers
import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/My Drive/Colab Notebooks/Assignment_3/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Now we will Read Train and Test Data


In [0]:
with open(datapath+"train.txt","r") as traintext:
  train_file=traintext.read()

with open(datapath+"test.txt","r") as testtext:
  test_file=testtext.read()


Pre_Processing data

In [0]:
remove_en=stopwords.words('english')
exception=['not','never', 'no','ever','very','nothing','really','extremely']


In [0]:
emodict = {0:'amusement', 1:'satisfaction', 2:'optimism', 3:'pride in achievement', 4:'contentment',
           5:'anger', 6:'fear', 7:'disgust', 8:'sadness', 9:'contempt'}
#emotion_label = {
#		"fear" : ['😳', '😰', '😨', '😬'],
#		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
#		"disgust" :['😭', '😩', '🤦', '😯', '🤢', '😖'],
#		"sadness" :['😫', '😭', '😞'] ,
#		"contempt" :['😔', '💔', '😏'],
#		"amusement" :['😲'],
#   "pride in achievement" :['👌', '👏', '👍'],
#    "satisfaction" :['😊', '🙏', '😔', '😎'],
#    "optimism" :['💪', '😎', '🛐'],
#    "contentment" :['😇', '😘', '😂', '😁', '💃'],
#    "none": ['none']  
#        }

emoji_label={
    "negative":['😳', '😰', '😨', '😬','😱', '😡', '😤', '😒', '😑', '😠','😭', '😩', '🤦', '😯', '🤢', '😖', '😫', '😭', '😞','😔', '💔', '😏' ],# fear+anger+disgust+sadness+contempt
    "neutral":['😊', '🙏'],#satisfaction
    "positive":['😇', '😘', '😂', '😁', '💃','💪', '😎', '🛐','👌', '👏', '👍','😲']#optimmism + contentment
}
emoji=['😳', '😰', '😨', '😬','😱', '😡', '😤', '😒', '😑', '😠','😭', '😩', '🤦', '😯', '🤢', '😖', '😫', '😭', '😞','😔', '💔', '😏', '😊', '🙏','😇', '😘', '😂', '😁', '💃','💪',
       '😎', '🛐','👌', '👏', '👍','😲']



In [6]:
!sudo pip install indic_transliteration -U
data = 'idm adbhutam'
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
print(transliterate(data, sanscript.HK, sanscript.DEVANAGARI))

     |████████████████████████████████| 102kB 3.6MB/s 
     |████████████████████████████████| 645kB 11.2MB/s 
     |████████████████████████████████| 911kB 48.9MB/s 
  Created wheel for splinter: filename=splinter-0.11.0-cp36-none-any.whl size=30144 sha256=d695f974a8d6182e3e35c95d4a8774ec75e735b6e8a941f2457678a722fa78db
  Stored in directory: /root/.cache/pip/wheels/9c/e1/35/4809427c48cb88853ddc1e701e9c5629bf192ae9324f2d0042
Successfully built splinter
इद्म् अद्भुतम्


In [8]:
train_id, train_text, train_label, train_c_h, train_c_e, train_c_o=[],[],[],[],[],[]

for tweet in train_file.split('\n\n'):
  lines=tweet.split('\n')
  try:
    train_label.append(lines[0].split()[2])
    train_id.append(lines[0].split()[1])
  except IndexError:
    del train_id[-1]
    continue
  temp_c,temp_c_h,temp_c_e,temp_c_o=[],[],[],[]
  for line in lines[1:]:
    words=line.split('\t')
    words[0]=words[0].lower()
    if(words[1]!='0'):
      words[0]=re.sub('[\W_]+','',words[0])
    if words[1]=='Eng' and words[0] in remove_en and words[0] not in exception:
      continue
    if 'http' in words[0]:
      continue
    temp_c.append(words[0])
    if words[1]== 'Eng':
      temp_c_e.append(words[0])
    if words[1]== 'Hin':
      temp_c_h.append(words[0])
    elif words[1]== 'O':
      if(words[0] in emoji):
        temp_c_o.append(words[0])
      
  if temp_c==[]:
    continue
  train_text.append(temp_c)
  train_c_h.append(temp_c_h)
  train_c_e.append(temp_c_e)
  train_c_o.append(temp_c_o)


print(len(train_text))
  
  
    










15131


In [9]:
test_id, test_text, test_label, test_c_h, test_c_e, test_c_o=[],[],[],[],[],[]

for tweet in test_file.split('\n\n'):
  lines=tweet.split('\n')
  try:
    test_label.append(lines[0].split()[2])
    test_id.append(lines[0].split()[1])
  except IndexError:
    del test_id[-1]
    continue
  temp_c,temp_c_h,temp_c_e,temp_c_o=[],[],[],[]
  for line in lines[1:]:
    words=line.split('\t')
    words[0]=words[0].lower()
    if(words[1]!='0'):
      #words[0]=re.sub('[\W_]+','',words[0])
      words[0]=words[0]
    if words[1]=='Eng' and words[0] in remove_en and words[0] not in exception:
      continue
    if 'http' in words[0]:
      continue
    temp_c.append(words[0])
    if words[1]== 'Eng':
      temp_c_e.append(words[0])
    if words[1]== 'Hin':
      temp_c_h.append(words[0])
    elif words[1]== 'O':
      if(words[0] in emoji):
        temp_c_o.append(words[0])
  if temp_c==[]:
    continue
  test_text.append(temp_c)
  test_c_h.append(temp_c_h)
  test_c_e.append(temp_c_e)
  test_c_o.append(temp_c_o)


print(len(test_text))
  
  

1869


In [10]:
train_tweet= [' '.join(i) for i in train_text]
test_tweet= [' '.join(i) for i in test_text]

train_tweet_dict, test_tweet_dict={},{}

train_tweet_dict['hin']= [' '.join(i) for i in train_c_h]
train_tweet_dict['eng']= [' '.join(i) for i in train_c_e]

test_tweet_dict['hin']= [' '.join(i) for i in test_c_h]
test_tweet_dict['eng']= [' '.join(i) for i in test_c_e]

train_tweet_dict['o']= [' '.join(i) for i in train_c_o]
test_tweet_dict['o']= [' '.join(i) for i in test_c_o]

np.unique(train_label)

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [82]:
train_tweet_dict['eng'][:10]

['pakistan israel ko',
 'co oxf8tr3bly',
 'co prnomskkn1',
 'atheist krishna jcb full trend',
 'loksabha bjp ko without co shtbwcb7fm',
 'noirnaveed angelahana6 cricketworldcup bhosdike',
 'bhaijan father son pic entire promotions mashallah s2xhwu6lud',
 'manojgajjar111 tumhara islea google co bxueug3xsn',
 'nolo weni weekend',
 'aimim sachins40805591 lage faad co gha9dwnz6u']

In [11]:
# MODEL LOADED- EMOJI2VEC and WORD2VEC
import gensim.models as gsm
e2v = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/emoji2vec.bin', binary=True)
model = gsm.KeyedVectors.load_word2vec_format('/content/gdrive/Shared drives/NLP/emoji_adding/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [125]:
# Model For hindi Embeddings
model_h = gsm.Word2Vec.load('/content/gdrive/My Drive/Colab Notebooks/Assignment_3/hi.bin')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [153]:
data ='bilkula'
data=sanscript.SCHEMES[sanscript.OPTITRANS].to_lay_indian(data)
word= transliterate(data, sanscript.HK, sanscript.DEVANAGARI)
model_h[word]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


KeyError: ignored

In [51]:
from numpy import *
negative_word_histogram,positive_word_histogram,neutral_word_histogram=[],[],[]
negative_tweet_e, positive_tweet_e, neutral_tweet_e=[],[],[]
negative_tweet_h, positive_tweet_h, neutral_tweet_h=[],[],[]
negative_tweet_o, positive_tweet_o, neutral_tweet_o=[],[],[]
train_tweet_embeddings=[] #eng+hindi+emoji
count = 0
tot = 0
for i in range(len(train_label)):
  temp=[]
  if(train_label[i] =="negative"):
  # NEGATIVE  
    tot += 1
    words_e = train_tweet_dict['eng'][i].split(" ")
    vector_list_e = [model[word] for word in words_e if word in model.vocab]
    j = 0
    while j < len(vector_list_e):
      if sum(np.isnan(vector_list_e[j]))>0:
        del vector_list_e[j]
      else:
        j += 1
    # vector_list_e = where(isnan(vector_list_e), 0, vector_list_e)
    # where_are_NaNs = isnan(vector_list_e)
    # vector_list_e[where_are_NaNs] = 0
    avg_vector_e = np.nanmean(vector_list_e,axis=0)
    if sum(np.isnan(avg_vector_e)) == 0:
      negative_tweet_e.append(avg_vector_e)

  
    



    words_h = train_tweet_dict['hin'][i].split(" ")
    vector_list_h = np.array([model[word] for word in words_h if word in model.vocab])
    vector_list_h = where(isnan(vector_list_h), 0, vector_list_h)
    where_are_NaNs = isnan(vector_list_h)
    vector_list_h[where_are_NaNs] = 0
    avg_vector_h = np.average(vector_list_h,axis=0)
    negative_tweet_h.append(avg_vector_h)
    
    words_o = train_tweet_dict['o'][i].split(" ")
    vector_list_o = np.array([e2v[word] for word in words_o if word in model.vocab])
    vector_list_o = where(isnan(vector_list_o), 0, vector_list_o)
    where_are_NaNs = isnan(vector_list_o)
    vector_list_o[where_are_NaNs] = 0
    avg_vector_o = np.average(vector_list_o,axis=0)
    negative_tweet_o.append(avg_vector_o)
    temp.append(avg_vector_e)
    temp.append(avg_vector_h)
    #temp.append(avg_vector_o)
    temp = np.average(temp,axis=0)
  if (train_label[i]=="neutral"):
    words_e = train_tweet_dict['eng'][i].split(" ")
    vector_list_e = np.array([model[word] for word in words_e if word in model.vocab])
    vector_list_e = where(isnan(vector_list_e), 0, vector_list_e)
    where_are_NaNs = isnan(vector_list_e)
    vector_list_e[where_are_NaNs] = 0
    avg_vector_e = np.average(vector_list_e,axis=0)
    neutral_tweet_e.append(avg_vector_e)
    
    words_h = train_tweet_dict['hin'][i].split(" ")
    vector_list_h = np.array([model[word] for word in words_h if word in model.vocab])
    vector_list_h = where(isnan(vector_list_h), 0, vector_list_h)
    where_are_NaNs = isnan(vector_list_h)
    vector_list_h[where_are_NaNs] = 0
    avg_vector_h = np.average(vector_list_h,axis=0)
    neutral_tweet_h.append(avg_vector_h)
    
    words_o = train_tweet_dict['o'][i].split(" ")
    vector_list_o = np.array([e2v[word] for word in words_o if word in model.vocab])
    vector_list_o = where(isnan(vector_list_o), 0, vector_list_o)
    where_are_NaNs = isnan(vector_list_o)
    vector_list_o[where_are_NaNs] = 0
    avg_vector_o = np.average(vector_list_o,axis=0)
    neutral_tweet_o.append(avg_vector_o)
    temp.append(avg_vector_e)
    temp.append(avg_vector_h)
    #temp.append(avg_vector_o)
    temp = np.average(temp,axis=0)

  if(train_label[i]=="positive"):
    words_e = train_tweet_dict['eng'][i].split(" ")
    vector_list_e = np.array([model[word] for word in words_e if word in model.vocab])
    vector_list_e = where(isnan(vector_list_e), 0, vector_list_e)
    where_are_NaNs = isnan(vector_list_e)
    vector_list_e[where_are_NaNs] = 0
    avg_vector_e = np.average(vector_list_e,axis=0)
    positive_tweet_e.append(avg_vector_e)
    
    words_h = train_tweet_dict['hin'][i].split(" ")
    vector_list_h = np.array([model[word] for word in words_h if word in model.vocab])
    vector_list_h = where(isnan(vector_list_h), 0, vector_list_h)
    where_are_NaNs = isnan(vector_list_h)
    vector_list_h[where_are_NaNs] = 0
    avg_vector_h = np.average(vector_list_h,axis=0)
    positive_tweet_h.append(avg_vector_h)
    
    words_o = train_tweet_dict['o'][i].split(" ")
    vector_list_o = np.array([e2v[word] for word in words_o if word in model.vocab])
    vector_list_o = where(isnan(vector_list_o), 0, vector_list_o)
    where_are_NaNs = isnan(vector_list_o)
    vector_list_o[where_are_NaNs] = 0
    avg_vector_o = np.average(vector_list_o,axis=0)
    positive_tweet_o.append(avg_vector_o)

    temp.append(avg_vector_e)
    temp.append(avg_vector_h)
    #temp.append(avg_vector_o)
    temp = np.average(temp,axis=0)
    
  train_tweet_embeddings.append(temp)



/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: Mean of empty slice


In [55]:
from numpy import *
tweet_ = {}
for label in ['negative', 'neutral', 'positive']:
  tweet_[label] = {}
  for key in ['eng', 'hin', 'o']:
    tweet_[label][key] = []

# negative_tweet_e, positive_tweet_e, neutral_tweet_e=[],[],[]
# negative_tweet_h, positive_tweet_h, neutral_tweet_h=[],[],[]
# negative_tweet_o, positive_tweet_o, neutral_tweet_o=[],[],[]
train_tweet_embeddings=[] #eng+hindi+emoji

for i in range(len(train_label)):
  temp=[]
  for key in ['eng', 'hin', 'o']:
    words = train_tweet_dict[key][i].split(" ")
    vector_list = [model[word] for word in words if word in model.vocab]
    j = 0
    while j < len(vector_list):
      if sum(np.isnan(vector_list[j]))>0:
        del vector_list[j]
      else:
        j += 1
    avg_vector = np.nanmean(vector_list,axis=0)
    if sum(np.isnan(avg_vector)) == 0:
      tweet_[train_label[i]][key].append(avg_vector)
    temp.append(avg_vector)
  temp = np.average(temp,axis=0)
  train_tweet_embeddings.append(temp)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: Mean of empty slice


In [58]:
tweet_['negative']['eng'])

4288

In [50]:
count, tot

(171, 4459)

In [34]:
# a = np.array([[1,1,1], [2,2,2],[3,3,3]]
#              )
# del a[0,:]
# a

ValueError: ignored

In [0]:
test_tweet_embeddings=[]
for i in range(len(test_tweet)):
  words = test_tweet[i].split(" ")
  vector_list = np.array([model[word] for word in words if word in model.vocab])
  avg_vector = np.average(vector_list,axis=0)
  test_tweet_embeddings.append(avg_vector)


In [176]:
len(train_tweet_embeddings[1])

300

In [102]:
train_tweet[:2]

[' adilnisarbutt pakistan ka ghra tauq pakistan israel ko tasleem nahein kerta isko palestine kehta he  occupied palestine',
 'madarchod mulle ye mathura me nahi dikha tha jab mullo ne hindu ko iss liye mara ki vo lasse ki paise mag liye    co  oxf8tr3bly']

In [61]:
avg_={}
for label in ['negative','neutral','positive']:
  avg_[label]={}
  for key in ['eng','hin','o']:
    avg_[label][key]=np.nanmean(tweet_[label][key],axis=0)
    print(sum(avg_[label][key]))


-0.35712773
-2.3616002
nan
-0.4074724
-2.0607688
nan
-0.297975
-1.7554432
nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [0]:
import scipy
similarity=[]
for i in range(len(train_tweet_embeddings)):
  temp=[]
  for label in ['negative','neutral','positive']:
    for key in ['eng','hin']:
      distance = scipy.spatial.distance.cosine(avg_[label][key],train_tweet_embeddings[i])
      temp.append(distance)
  similarity.append(temp)

In [0]:
similarity_test=[]
for i in range(len(test_tweet_embeddings)):
  temp=[]
  for label in ['negative','neutral','positive']:
    for key in ['eng','hin']:
      distance = scipy.spatial.distance.cosine(avg_[label][key],test_tweet_embeddings[i])
      temp.append(distance)
  similarity_test.append(temp)

In [66]:
similarity=np.array(similarity)
similarity_test=np.array(similarity_test)

similarity.shape, similarity_test.shape



((15131, 6), (1869, 6))

In [0]:
train_final = []
add_shape = similarity[0].shape
train_indexes = []
new_Y_train = []
for i in range(len(similarity)):
  if (np.sum(np.isnan(similarity[i,:])) == 0):
    train_final.append(similarity[i])
    new_Y_train.append(Y_train[i])
  else:
    train_indexes.append(i)

test_final = []
add_shape = similarity_test[0].shape
test_indexes = []
new_Y_test = []
for i in range(len(similarity_test)):
  if (np.sum(np.isnan(similarity_test[i,:])) == 0):
    test_final.append(similarity_test[i])
    new_Y_test.append(Y_test[i])
  else:
    test_indexes.append(i)


In [75]:
np.array(train_final).shape
len(train_indexes)

15131

In [73]:
from sklearn.tree import DecisionTreeClassifier

model=DecisionTreeClassifier()
model.fit(train_final,new_Y_train)

ValueError: ignored

In [0]:
np_train_tweet_embeddings = np.zeros((len(train_tweet_embeddings), 300))

for i in range(len(train_tweet_embeddings)):
  np_train_tweet_embeddings[i,:] = np.array(train_tweet_embeddings[i])

np_train_tweet_embeddings.shape

np_test_tweet_embeddings = np.zeros((len(test_tweet_embedding), 300))

for i in range(len(test_tweet_embedding)):
  np_test_tweet_embeddings[i,:] = np.array(test_tweet_embedding[i])

np_train_tweet_embeddings.shape
np_test_tweet_embeddings.shape

In [0]:
# AVG HINDI+ENGLISH - P, N, NU
avg_positive_tweet=[]
avg_positive_tweet.append(positive_tweet_e)
avg_positive_tweet.append(positive_tweet_h)
avg_positive_tweet= np.average(avg_positive_tweet, axis=0)

avg_negative_tweet=[]
avg_negative_tweet.append(negative_tweet_e)
avg_negative_tweet.append(negative_tweet_h)
avg_negative_tweet= np.average(avg_negative_tweet, axis=0)


avg_neutral_tweet=[]
avg_neutral_tweet.append(neutral_tweet_e)
avg_neutral_tweet.append(neutral_tweet_h)
avg_neutral_tweet= np.average(avg_neutral_tweet, axis=0)

In [16]:
#print(type(train_tweet_embeddings))
#print(type(train_tweet_embeddings[0]))
np_train_tweet_embeddings = np.zeros((len(train_tweet_embeddings), 300))

for i in range(len(train_tweet_embeddings)):
  np_train_tweet_embeddings[i,:] = np.array(train_tweet_embeddings[i])

np_train_tweet_embeddings.shape

np_test_tweet_embeddings = np.zeros((len(test_tweet_embedding), 300))

for i in range(len(test_tweet_embedding)):
  np_test_tweet_embeddings[i,:] = np.array(test_tweet_embedding[i])

np_train_tweet_embeddings.shape
np_test_tweet_embeddings.shape


(1869, 300)

Now we will further analyse the new Train and Test Data

In [18]:
label_v = {'negative':0, 'neutral':1, 'positive':2}
y_train = np.array([label_v[i] for i in train_label])
y_test = np.array([label_v[i] for i in test_label])

max_features = 20000
tokenizer1 = Tokenizer(num_words=max_features)
tokenizer1.fit_on_texts(train_tweet)

max_len = 300
num_classes = 3

sequences_train = tokenizer1.texts_to_sequences(train_tweet)
sequences_test = tokenizer1.texts_to_sequences(test_tweet)
X_train = sequence.pad_sequences(sequences_train, maxlen=max_len)
X_test = sequence.pad_sequences(sequences_test, maxlen=max_len)
X_train_1=np.hstack((X_train, np_train_tweet_embeddings))
X_test_1=np.hstack((X_test, np_test_tweet_embeddings))

#X_train=append(X_train_1,train_tweet_embeddings, axis=1)
  
#X_test=append(X_test_1,test_tweet_embedding, axis=1)
  
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

print('X_train shape:', X_train_1.shape)
print('X_test shape:', X_test_1.shape)


X_train shape: (15131, 600)
X_test shape: (1869, 600)


In [0]:

nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


cmodel1 = Sequential()
cmodel1.add(Dense(600))
cmodel1.add(Dropout(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
#cmodel1.add(Convolution1D(nb_filter=nb_filter,
#                       filter_length=filter_length,
#                       border_mode='valid',
#                       activation='tanh',
#                       subsample_length=1))

#cmodel1.add(BatchNormalization())
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)

#cmodel1.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel1.add(Dense(hidden_dims))
cmodel1.add(Dropout(0.2))
cmodel1.add(Activation('relu'))
cmodel1.add(Dense(num_classes))
cmodel1.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel1.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

In [31]:
cmodel1.fit(X_train_1, Y_train, epochs = 2)

Epoch 1/2
15131/15131 [==============================] - 6s 369us/step - loss: nan - acc: 0.0022
Epoch 2/2
15131/15131 [==============================] - 5s 353us/step - loss: nan - acc: 0.0000e+00


In [32]:
predsc1 = cmodel1.predict_classes(X_test_1, verbose=0)
np.sum(predsc1==y_test)/len(y_test)

0.28517924023542

In [61]:
from sklearn.metrics import precision_recall_fscore_support as p_recall_f
p_recall_f(y_test, predsc1, average='micro')

(0.5564472980203318, 0.5564472980203318, 0.5564472980203318, None)

In [62]:

p_recall_f(y_test, predsc1, average='macro')

(0.5570606144166599, 0.5709277653907551, 0.5591825497916824, None)

In [63]:
p_recall_f(y_test, predsc1, average='weighted')

(0.55634354185309, 0.5564472980203318, 0.551201611107008, None)

In [64]:

cmodel1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         6000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 300)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 300)         270300    
_________________________________________________________________
lambda_1 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
_________________________________________________________________
activation_9 (Activation)    (None, 300)              